In [2]:
#!pip install tweepy
import pandas as pd
import tweepy

# Scraping

In [1]:
info_api_key = "fSFhW6NJHivhKs88VLtlKQ5t6"
info_api_key_secret = "X7mVc0epNqxlCdRpa4pP2ymRdpbZwnrEr1edW8SG7JhwMgtEnA"
info_bearer_token = "AAAAAAAAAAAAAAAAAAAAAGyulQEAAAAAg7rYt7A8KD2yYFu0datNWtadXOU%3Dou4oAmL2wFiZxrckLd5OCHioEDDRJw4ZEDp7AuuJx7WHIENpGq"

info_client_id = "VzlNTVNlYmw4b21ORUtKdlJiYkc6MTpjaQ"
info_client_secret = "RhKKHZ0bHeq1QqqNsO3Kzw2_dlo0J1strmd9C0n1bziPwn9PiS"

info_acces_token = "1574148627672571904-bIjrsPyaC3zLKaYuF9t7rh4PGkPtwT"
info_acces_token_secret = "jknxWvirBnliMgzINTmmOvGZftZnoShpJe4x1seNBdfth"

info_callback_uri = "https://wibbo.org"
info_website_url = "https://wibbo.org"

In [319]:
def Recuperation_tweets(L_mots,nb):
    client = tweepy.Client(bearer_token=info_bearer_token)

    query = ''
    for m in L_mots:
        query += m + ' OR '
    query=query[:-3]
    query+='lang=en'
    

    #tweets = client.search_recent_tweets(query,max_results=nb,tweet_fields=['context_annotations', 'created_at'], 
                                         #expansions=['author_id'])
    tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
                              tweet_fields=['context_annotations', 'created_at'], expansions=['author_id'],
                              max_results=100).flatten(limit=nb)
    L_tweets=[]
    for tweet in tweets:
        #if len(tweet.context_annotations) > 0: #s'il y a un domaine et l'entité
        L_tweets.append([tweet.data['id'], tweet.data['text'], tweet.data['author_id'], tweet.data['created_at']])
    
    return L_tweets


In [322]:
L=Recuperation_tweets(["suicide","suicidal","I want to die","help","kill myself",],10000)
df=pd.DataFrame(L,columns=['id','text','auteur','date'])


In [327]:
#df.to_csv('Tweepy_Suicide_Tweets.csv',sep=';')
#df.to_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Tweepy_Suicide_Tweets.csv',sep=';',index=False)


# Analyse Users

## Histrorique des tweets des personnes à risques

In [3]:
df_init=pd.read_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Tweepy_Suicide_Tweets_Final.csv',sep=';')
df=df_init.copy()
df.head()

,id,text,auteur,date,keyword_label,valder_label,textbloc_polarity_label,textbloc_subjectivity_label,happy_transformer_label,bert_label,suicidal_bert_label,polarity_modele,modele_groupement_score,groupement_all_modele_par_3
0,1618391698639556610,https t co VgRPNEqwaN,813273195851681792,2023-01-25T23:33:43.000Z,1,0,0.000000,0.000000,1,0,0,1,2,0
1,1618391687990247424,RT I was today years old when I found out Ro...,808471634768265216,2023-01-25T23:33:41.000Z,0,1,0.050000,0.100000,1,1,0,1,3,1
2,1618391665726877697,RT Another day another domestic abuser in Am...,105294403,2023-01-25T23:33:36.000Z,1,1,0.216667,0.466667,1,0,0,1,3,1
3,1618391659888377856,RT NO SHE DIDNT BUT WAITING ROOM TAKEN OFF SP...,1332277646764482560,2023-01-25T23:33:34.000Z,1,1,0.416667,0.583333,1,1,0,1,4,1
4,1618391653571784704,Your effectively can t take away the right of ...,29817415,2023-01-25T23:33:33.000Z,0,1,0.442857,0.667857,1,1,0,1,3,1


In [26]:
L_personnes_a_risques=df[df['groupement_all_modele_par_3']==1]['auteur'].to_list()
L_personnes_a_risques=list(set(L_personnes_a_risques))#garder les valeurs uniques
print(len(L_personnes_a_risques))
print(L_personnes_a_risques[:10])

7110
[808471634768265216, 1555056913272176641, 701824658610454530, 1606765758851186691, 821369421360791556, 1247896418871738372, 1303349962856595461, 1066571247473983494, 1380877363530104840, 1607932020910325769]


In [56]:
def Recuperation_tweets_personnes(L_personnes,nb_tweets):
    client = tweepy.Client(bearer_token=info_bearer_token)

    L_tweets=[]
    for p_id in L_personnes:
        tweets = tweepy.Paginator(client.get_users_tweets,id=p_id,tweet_fields=['context_annotations', 'created_at'],
                                  expansions=['author_id'],max_results=100).flatten(limit=nb_tweets)
        
        #return tweets
        
        if tweets != None:
            for tweet in tweets:
            #if len(tweet.context_annotations) > 0: #s'il y a un domaine et l'entité
                L_tweets.append([tweet.data['id'], tweet.data['text'], tweet.data['author_id'], tweet.data['created_at']])
    
    return L_tweets

In [196]:
L=Recuperation_tweets_personnes(L_personnes_a_risques[:5],1)
L

[['1618443251454992384',
  'I’m mad I didn’t get to save that video 😒',
  '808471634768265216',
  '2023-01-26T02:58:35.000Z'],
 ['1618391987073449984',
  'RT @SunitaInsan1982: #MSGBhandara \n•Millions of people reached Sirsa Darbar,Haryana.\n•Many people gave up drugs,get method of meditation by…',
  '1555056913272176641',
  '2023-01-25T23:34:52.000Z'],
 ['1618591787580522496',
  '@hamadux01 Ah',
  '701824658610454530',
  '2023-01-26T12:48:48.000Z'],
 ['1618410217934565377',
  '@DavidBlue https://t.co/bjjTnysfun',
  '1606765758851186691',
  '2023-01-26T00:47:19.000Z'],
 ['1618502564231925760',
  "RT @revolutionvote: The left ruins things... They are like oh, you're ok with drag queens? Well fuck it, we'll get them to wave their wangs…",
  '821369421360791556',
  '2023-01-26T06:54:16.000Z']]

In [180]:
len(L_personnes_a_risques)

7110

## Infos des profils

In [201]:
def Infos_profils(L_users_id):
    client = tweepy.Client(bearer_token=info_bearer_token)
    L_tot=[]
    for i in range(0,len(L_users_id),100):
        a=i
        b=a+100

        users=client.get_users(ids=L_users_id[a:b], user_fields= ['created_at','description','id','location', 
                                                                    'name','protected','public_metrics','username',
                                                                         'verified','verified_type'])
        L_tot.append(users)
                
    L_info_users=[]
    for users in L_tot:
        for user in users.data:
            L_info_users.append([user['id'],user['name'],user['username'],user['created_at'],user['location'],
                                 user['public_metrics']['followers_count'],user['public_metrics']['following_count'],
                                 user['public_metrics']['tweet_count'],user['public_metrics']['listed_count'],
                                 user['protected'],user['verified'],user['verified_type'],user['description']])

            
    df_users=pd.DataFrame(L_info_users,columns=['id','name','username','created_at','location','followers_count',
                                                'following_count','tweet_count','listed_count','protected','verified',
                                                'verified_type','description'])
    
    return df_users
    

In [203]:
df_users=Infos_profils(L_personnes_a_risques)
df_users.head(10)

,id,name,username,created_at,location,followers_count,following_count,tweet_count,listed_count,protected,verified,verified_type,description
0,808471634768265216,𝓡𝓮𝓮𝓶𝓪💕,_ThugAngel25,2016-12-13 00:40:33+00:00,989,2970,2566,38702,2,False,False,none,"Like the night. without the dark, we'd never s..."
1,1555056913272176641,Sushila Beniwal,SushilaBeniwal3,2022-08-04 05:04:19+00:00,None,26,26,7313,0,False,False,none,
2,701824658610454530,Ramadan ‎ރ,shawarma_djej,2016-02-22 17:43:32+00:00,🇫🇷,259,717,7697,13,False,False,none,🇱🇧🇲🇦🇨🇮
3,1606765758851186691,Steve Long,hobbyanimations,2022-12-24 21:36:46+00:00,None,2170,52,89,0,False,False,none,Disabled Veteran who as a hobby makes Films a...
4,821369421360791556,Right of Revolution,revolutionvote,2017-01-17 14:51:45+00:00,None,1849,1769,81473,7,False,False,none,I identify as an empire builder. My preferred ...
5,1247896418871738372,Teg ! ／(≧ x ≦)＼SSD 📌,USHANKAIVAN,2020-04-08 14:38:24+00:00,They/them - wasian - 15,615,903,26047,10,False,False,none,subscribe and like this vid to see me cry abt ...
6,1303349962856595461,IndyWales-Anti Westminster 🇪🇺🏴󠁧󠁢󠁷󠁬󠁳󠁿🇺🇦,TLlanelli,2020-09-08 15:10:33+00:00,Wales,2726,5002,163727,20,False,False,none,Your ID: #Welsh #English #Scottish or British ...
7,1066571247473983494,Jess ⚢ ☭ ⛓,dykemelodrama,2018-11-25 05:56:16+00:00,"San Francisco, CA",385,324,37412,2,False,False,none,"Jess, 29 🔞🏳️‍🌈🏳️‍⚧️⛓ ☭🏴 Disco loving Dyke, wan..."
8,1380877363530104840,Labii𓂀,Lab1_S,2021-04-10 13:36:51+00:00,Vranjevc,436,366,29638,2,False,False,none,i retweet weird shit and shitpost
9,1607932020910325769,Jeff Cox,TaterHater16,2022-12-28 02:51:39+00:00,"Ripley, TN",63,85,1629,0,False,False,none,"Happily Married to Neda Cox, Father to the gre..."


In [204]:
df_users.to_csv(r'C:\Users\owen9\OneDrive\Documents\GitHub\TwitterFeelingAnalyse\Dataset\Infos_users.csv',sep=';',index=False)
                

# Envoyer message

In [ ]:
client = tweepy.Client(bearer_token=info_bearer_token)

In [16]:
screen_name = "axelg99"
twitterid = client.get_user(username=screen_name)
twitterid.data.id

3080430177

In [20]:
message="Si tu as des pensées suicidaires, s'il te plaît contacte immédiatement SOS Amitié pour obtenir de l'aide. Les horaires sont disponibles 24h/24 et 7j/7. Pour plus d'informations, compose le numéro 09 72 39 40 50. Tu n'es pas seul, nous sommes là pour t'aider."


In [21]:
api.send_direct_message(twitterid.data.id, message)

DirectMessage(_api=<tweepy.api.API object at 0x00000273EDBDC280>, _json={'type': 'message_create', 'id': '1618756037703499780', 'created_timestamp': '1674776489078', 'message_create': {'target': {'recipient_id': '3080430177'}, 'sender_id': '1574148627672571904', 'message_data': {'text': "Si tu as des pensées suicidaires, s'il te plaît contacte immédiatement SOS Amitié pour obtenir de l'aide. Les horaires sont disponibles 24h/24 et 7j/7. Pour plus d'informations, compose le numéro 09 72 39 40 50. Tu n'es pas seul, nous sommes là pour t'aider.", 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}}}}, type='message_create', id='1618756037703499780', created_timestamp='1674776489078', message_create={'target': {'recipient_id': '3080430177'}, 'sender_id': '1574148627672571904', 'message_data': {'text': "Si tu as des pensées suicidaires, s'il te plaît contacte immédiatement SOS Amitié pour obtenir de l'aide. Les horaires sont disponibles 24h/24 et 7j/7. Pour plus d'i

# Tests

In [16]:
auth = tweepy.OAuth2BearerHandler(info_bearer_token)
api = tweepy.API(auth)
api

In [90]:
auth = tweepy.OAuth2AppHandler(info_api_key, info_api_key_secret)
api = tweepy.API(auth)
api

In [89]:
auth = tweepy.OAuth2UserHandler(info_api_key, info_api_key_secret, info_acces_token, info_acces_token_secret)
api = tweepy.API(auth)
api

TypeError: __init__() takes 1 positional argument but 5 were given

In [91]:
client = tweepy.Client(info_bearer_token)
client

In [66]:
client.create_tweet(in_reply_to_tweet_id=twitterid[0].id, text='test')

TypeError: Consumer key must be string or bytes, not NoneType

In [22]:
client = tweepy.Client(info_api_key,info_api_key_secret,info_acces_token,info_acces_token_secret)
client

In [93]:
oauth2_user_handler = tweepy.OAuth2UserHandler(
    client_id=info_client_id,
    redirect_uri=info_callback_uri,
    scope=info_scope,
    # Client Secret is only necessary if using a confidential client
    client_secret=info_client_secret
)

print(oauth2_user_handler.get_authorization_url())

https://twitter.com/i/oauth2/authorize?response_type=code&client_id=VzlNTVNlYmw4b21ORUtKdlJiYkc6MTpjaQ&redirect_uri=https%3A%2F%2Ftwitter.com%2F&scope=tweet.read+tweet.write+users.read&state=fE1kMMnR54NV0jZGMNveddWqQvVXzc&code_challenge=3wWCN2wK6T9iFvK4NqWauFq76aG4NJoqqmvJDi8zjjA&code_challenge_method=S256


In [62]:
auth_url='https://twitter.com/home?state=lJ59K5uCQminldrbNR5OX7QmePFTya&code=czRUZEIyeWV4Q2FKR2ZKemdqa1lxei1SdEpLSV8zUmxGTWU1ZkgyaG1iU24tOjE2NzQ2NjM3Nzc1MDA6MToxOmFjOjE'
access_token = oauth2_user_handler.fetch_token(auth_url)

In [63]:
api = tweepy.API(auth)
api

In [ ]:
client = tweepy.Client(bearer_token=info_bearer_token)

# Create a tweet query
# https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/5-how-to-write-search-queries.md
query = 'suicide lang:en'
tweets = client.search_recent_tweets(query, max_results=100)
for tweet in tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print(tweet.context_annotations)

In [41]:
client = tweepy.Client(access_token)
client

In [7]:
# Authenticate access
auth = tweepy.OAuthHandler(info_api_key, info_api_key_secret)
auth.set_access_token(info_acces_token, info_acces_token_secret)
# Create API handler
api = tweepy.API(auth)

api
client = tweepy.Client(bearer_token=info_bearer_token)

In [70]:
twts = api.search(q="Hello World!")

#list of specific strings we want to check for in Tweets
t = ['Hello world!',
    'Hello World!',
    'Hello World!!!',
    'Hello world!!!',
    'Hello, world!',
    'Hello, World!']

for s in twt:
    for i in t:
        if i == s.text:
            sn = s.user.screen_name
            m = "@%s Hello!" % (sn)
            s = api.update_status(m, s.id)

AttributeError: 'API' object has no attribute 'search'

In [72]:
recipient_id = "1618042845625212928"  # ID of the user
api.send_direct_message(recipient_id, "Hoi hoi")

Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve

In [96]:
info_api_key = "fSFhW6NJHivhKs88VLtlKQ5t6"
info_api_key_secret = "X7mVc0epNqxlCdRpa4pP2ymRdpbZwnrEr1edW8SG7JhwMgtEnA"
info_bearer_token = "AAAAAAAAAAAAAAAAAAAAAGyulQEAAAAAg7rYt7A8KD2yYFu0datNWtadXOU%3Dou4oAmL2wFiZxrckLd5OCHioEDDRJw4ZEDp7AuuJx7WHIENpGq"

info_client_id = "VzlNTVNlYmw4b21ORUtKdlJiYkc6MTpjaQ"
info_client_secret = "RhKKHZ0bHeq1QqqNsO3Kzw2_dlo0J1strmd9C0n1bziPwn9PiS"

info_acces_token = "1574148627672571904-bIjrsPyaC3zLKaYuF9t7rh4PGkPtwT"
info_acces_token_secret = "jknxWvirBnliMgzINTmmOvGZftZnoShpJe4x1seNBdfth"

info_callback_uri = "https://wibbo.org"
info_website_url = "https://wibbo.org"

info_scope = ["tweet.read","tweet.write","users.read"]

oauth2_user_handler = tweepy.OAuth2UserHandler(
    client_id= info_client_id,
    redirect_uri= info_callback_uri,
    scope= info_scope,
    
    # Client Secret is only necessary if using a confidential client
    client_secret= info_client_secret
)

print(oauth2_user_handler.get_authorization_url())
state = input("Tapez l'url : ")
access_token = oauth2_user_handler.fetch_token(state)
client = tweepy.Client(access_token)

https://twitter.com/i/oauth2/authorize?response_type=code&client_id=VzlNTVNlYmw4b21ORUtKdlJiYkc6MTpjaQ&redirect_uri=https%3A%2F%2Fwibbo.org&scope=tweet.read+tweet.write+users.read&state=fuBMvSWW9qU5lHQTfKPYu4lqTCbxtF&code_challenge=g3gO5zyVAzigXWtRjQ4_z2C_I8IPaNbCW2ZoUsD2xzI&code_challenge_method=S256
Tapez l'url : https://wibbo.org/?state=fuBMvSWW9qU5lHQTfKPYu4lqTCbxtF&code=WDJwNDVNZjBxSTRsSXlGSjZIU2U1MXVHQmZtV3lOVmw0Q2NiM25panMxLW1UOjE2NzQ2ODM5MTUwNTg6MToxOmFjOjE


# Tweets d'un user

In [111]:
tweets = client.get_users_tweets("1059331200261767168")
for tweet in tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print(tweet.context_annotations)

RT @PornoxDia: Chanel Preston #StarCards💫
👉https://t.co/1Q7PU9bzsS https://t.co/fcwr0i5KID
RT @PornoxDia: Dillion Harper #StarCards💫
👉https://t.co/qtC3zBLYZA https://t.co/5XWNuM9N5F
RT @PornoxDia: Shawna Lenee #StarCards💫
👉https://t.co/ErKRPCtI53 https://t.co/xEBEzk3TcD
RT @PornoxDia: Julie Cash #StarCards💫
👉https://t.co/tZo7FphUm2 https://t.co/oa7eYE1Vdd
RT @PornoxDia: Ryan Conner #StarCards💫
👉https://t.co/VSI1k7C69C https://t.co/unJXYBuMKP
RT @TeamDanaRico: Soy muy versátil, me gusta bromear, coquetear y especialmente ser travieso y juguetón en mis shows, así que asegúrese de…
RT @TeamDanaRico: ES HORA DE MOSTRAR MI MEJOR PARTE encontraras una chica lista para explorar y fantasear contigo https://t.co/hPzZbuJdh6 h…
RT @TeamDanaRico: El hidrogeno más el oxígeno producen agua, tu y yo producimos placer. https://t.co/eoLTYCv1FZ https://t.co/ZR97vXe6Jx
RT @TeamDanaRico: Ven y disfruta conmigo de interesantes aventuras que te harán perder la cabeza. https://t.co/NdI1zOt9hU https://t.co/sL